In [ ]:
import cv2
import numpy as np

def hysteresis_threshold(image):
    lowThreshold=70
    highThreshold=90
    
    M, N = image.shape
    result = np.zeros((M,N), dtype=np.float32)

    for i in range(1,M-1):
      for j in range(1,N-1):
        if image[i][j] >= highThreshold:
          result[i][j] = 1
        elif image[i][j] < lowThreshold:
          result[i][j] = 0
        else:
          result[i][j] = 0
          for m in range(i-1, i+2):
            for n in range(j-1, j+2):
              if result[m][n] == 1 or image[m][n] >= highThreshold:
                result[i][j] = 1
                break
    
    return result




def get_proposals_for_train(img,l,IMG_WIDTH = 200,IMG_HEIGHT = 200):

  sobel_filter = np.array([[-1,0,1],
                           [-2,0,2],
                           [-1,0,1]],dtype=np.float32)
  open1SE = np.ones((5,1),np.uint8)
  closeSE = np.ones((1,23),np.uint8)
  open2SE = np.ones((1,20),np.uint8)

  label = []
  proposals = []

  blur = cv2.bilateralFilter(img,5,75,75)

  gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
  
  clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(10,10))
  gray = clahe.apply(gray)

  sobel = cv2.filter2D(np.float32(gray),-1,sobel_filter)
  abs = np.absolute(sobel)
  thresh = hysteresis_threshold(abs)
  open1 = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, open1SE)
  close = cv2.morphologyEx(open1, cv2.MORPH_CLOSE, closeSE)
  open2 = cv2.morphologyEx(close, cv2.MORPH_OPEN, open2SE)
  open2 = np.array(open2.tolist(),dtype= np.uint8)

  cnts,_ = cv2.findContours(open2.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  cnts = sorted(cnts, key=len,reverse=True) 
  
  if len(cnts)>0:
    x1,y1,w1,h1 = cv2.boundingRect(cnts[0])
    
    if len(cnts)>1:

      area = cv2.contourArea(cnts[0])
      rect_area = w1*h1
      n1 = area/rect_area

      x2,y2,w2,h2 = cv2.boundingRect(cnts[1])
      area = cv2.contourArea(cnts[1])
      rect_area = w2*h2
      n2 = area/rect_area

      if n1 > n2 or w2<0.6*w1 or h2<0.4*h1:
        resized = cv2.resize(img[y1:y1+h1+1 , x1:x1+w1+1], (IMG_WIDTH,IMG_HEIGHT), interpolation = cv2.INTER_AREA)
        proposals.append(resized)
        label.append(l)

      else:
        resized = cv2.resize(img[y2:y2+h2+1 , x2:x2+w2+1], (IMG_WIDTH,IMG_HEIGHT), interpolation = cv2.INTER_AREA)
        proposals.append(resized)
        label.append(l)

    else:
      resized = cv2.resize(img[y1:y1+h1+1 , x1:x1+w1+1], (IMG_WIDTH,IMG_HEIGHT), interpolation = cv2.INTER_AREA)
      proposals.append(resized)
      label.append(l)

  else:
    if l != 0:
      resized = cv2.resize(img, (IMG_WIDTH,IMG_HEIGHT), interpolation = cv2.INTER_AREA)
      proposals.append(resized)
      label.append(l)

  return proposals,label




def get_proposals_for_test(img):
  IMG_WIDTH = 200
  IMG_HEIGHT = 200

  sobel_filter = np.array([[-1,0,1],
                           [-2,0,2],
                           [-1,0,1]],dtype=np.float32)
  open1SE = np.ones((5,1),np.uint8)
  closeSE = np.ones((1,23),np.uint8)
  open2SE = np.ones((1,20),np.uint8)

  proposals = []

  blur = cv2.bilateralFilter(img,5,75,75)

  gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
  
  clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(10,10))
  gray = clahe.apply(gray)

  sobel = cv2.filter2D(np.float32(gray),-1,sobel_filter)
  abs = np.absolute(sobel)
  thresh = hysteresis_threshold(abs)
  open1 = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, open1SE)
  close = cv2.morphologyEx(open1, cv2.MORPH_CLOSE, closeSE)
  open2 = cv2.morphologyEx(close, cv2.MORPH_OPEN, open2SE)
  open2 = np.array(open2.tolist(),dtype= np.uint8)
  
  cnts,_ = cv2.findContours(open2.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  
  cnts = sorted(cnts, key=len,reverse=True) 

  if len(cnts)>0:
    x1,y1,w1,h1 = cv2.boundingRect(cnts[0])
    
    if len(cnts)>1:
      area = cv2.contourArea(cnts[0])
      rect_area = w1*h1
      n1 = area/rect_area

      x2,y2,w2,h2 = cv2.boundingRect(cnts[1])
      area = cv2.contourArea(cnts[1])
      rect_area = w2*h2
      n2 = area/rect_area

      if n1 > n2 or w2<0.6*w1 or h2<0.4*h1 :
        resized = cv2.resize(img[y1:y1+h1+1 , x1:x1+w1+1], (IMG_WIDTH,IMG_HEIGHT), interpolation = cv2.INTER_AREA)
        proposals.append(resized)

      else:
        resized = cv2.resize(img[y2:y2+h2+1 , x2:x2+w2+1], (IMG_WIDTH,IMG_HEIGHT), interpolation = cv2.INTER_AREA)
        proposals.append(resized)

    else:
      resized = cv2.resize(img[y1:y1+h1+1 , x1:x1+w1+1], (IMG_WIDTH,IMG_HEIGHT), interpolation = cv2.INTER_AREA)
      proposals.append(resized)

  else:
    resized = cv2.resize(img, (IMG_WIDTH,IMG_HEIGHT), interpolation = cv2.INTER_AREA)
    proposals.append(resized)
  

  return proposals